In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_hist_gradient_boosting  # Permet d'activer HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

In [32]:
df = pd.read_csv('data/allocine_silver_2.csv')

In [33]:

# Renommer les colonnes 'réalisateur' et 'Entrées_1ère_semaine'
df.rename(columns={'réalisateur': 'realisateur', 'Entrées_1ère_semaine': 'Entrees_1ere_semaine'}, inplace=True)


In [34]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import FunctionTransformer
import numpy as np


 # Séparer les colonnes catégorielles et numériques
categorical_features = ["acteur_1", "acteur_2", "acteur_3", "realisateur", "distributeur", "genre", "pays"]
numeric_features = ["duree", "nominations", "prix", "annee_production"]



# Créer les transformers pour les colonnes catégorielles et numériques
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numeric_transformer = RobustScaler()

# Mettre à jour le preprocesseur pour inclure la transformation logarithmique
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numeric_features)
    ]
)


In [35]:

# Diviser les données en caractéristiques (X) et la variable cible (y)
X = df[["acteur_1", "acteur_2", "acteur_3", "realisateur", "distributeur", "genre", "pays", "duree", "nominations", "prix", "annee_production"]] # Caractéristiques (toutes les colonnes sauf 'Entrées_1ère_semaine')
y = df['Entrees_1ere_semaine']  # Variable cible

# Diviser les données en jeux d'entraînement (80%) et de test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=32)

In [36]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor


# Créer le pipeline complet avec le nouveau modèle RandomForestRegressor
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())  
])

# Entraîner le modèle sur les données d'entraînement
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['acteur_1', 'acteur_2',
                                                   'acteur_3', 'realisateur',
                                                   'distributeur', 'genre',
                                                   'pays']),
                                                 ('num', RobustScaler(),
                                                  ['duree', 'nominations',
                                                   'prix',
                                                   'annee_production'])])),
                ('regressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_byle...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

In [37]:
model.score(X_test, y_test)

0.5440004811446799

In [38]:
import pickle

# Enregistrez le meilleur modèle dans un fichier pickle
with open('test_model_jo.pkl', 'wb') as file:
    pickle.dump(model, file)
